In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('train.csv')

In [2]:
##import os

##os.chdir("")

In [3]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) 
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [4]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 1000)

Iteration:  0
[1 1 1 ... 3 4 7] [6 3 1 ... 9 4 2]
0.12221951219512195
Iteration:  10
[1 3 1 ... 3 4 7] [6 3 1 ... 9 4 2]
0.1951219512195122
Iteration:  20
[1 3 1 ... 3 4 7] [6 3 1 ... 9 4 2]
0.24817073170731707
Iteration:  30
[1 3 1 ... 3 4 7] [6 3 1 ... 9 4 2]
0.28192682926829266
Iteration:  40
[1 3 1 ... 3 4 7] [6 3 1 ... 9 4 2]
0.3118292682926829
Iteration:  50
[1 3 1 ... 3 4 2] [6 3 1 ... 9 4 2]
0.3444634146341463
Iteration:  60
[2 3 1 ... 0 4 2] [6 3 1 ... 9 4 2]
0.38846341463414635
Iteration:  70
[2 3 1 ... 0 4 2] [6 3 1 ... 9 4 2]
0.42317073170731706
Iteration:  80
[2 3 1 ... 0 4 2] [6 3 1 ... 9 4 2]
0.4511219512195122
Iteration:  90
[2 3 1 ... 9 4 2] [6 3 1 ... 9 4 2]
0.4766341463414634
Iteration:  100
[2 3 1 ... 9 4 2] [6 3 1 ... 9 4 2]
0.49990243902439024
Iteration:  110
[2 3 1 ... 9 4 2] [6 3 1 ... 9 4 2]
0.5212682926829268
Iteration:  120
[2 3 1 ... 9 6 2] [6 3 1 ... 9 4 2]
0.5460975609756098
Iteration:  130
[2 3 1 ... 9 6 2] [6 3 1 ... 9 4 2]
0.5696341463414634
Iteration: 

In [7]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[6 9 3 3 6 3 4 6 0 5 4 7 1 5 8 3 6 7 4 8 5 6 2 6 7 1 6 0 2 9 0 5 2 6 2 1 6
 2 3 0 1 6 8 4 3 0 6 9 4 7 0 9 4 3 0 2 1 6 8 5 4 7 5 9 4 4 9 7 4 8 9 3 4 6
 7 9 5 9 4 0 0 8 8 9 4 5 1 8 9 2 8 4 9 4 7 1 6 5 4 3 1 7 6 2 8 0 6 2 4 2 8
 2 6 0 9 3 2 9 6 3 7 6 0 0 5 3 2 5 9 1 4 4 0 2 1 1 9 3 5 8 9 8 9 3 3 2 0 2
 0 2 2 5 8 3 9 1 6 1 2 3 9 8 3 4 9 2 5 4 9 1 7 4 8 1 2 4 6 2 3 2 4 0 4 6 8
 2 4 1 6 1 6 2 7 1 4 3 6 3 5 5 8 8 0 6 5 3 3 7 5 2 5 2 1 8 4 1 1 3 6 3 6 4
 7 1 4 9 2 4 4 5 2 1 2 3 4 9 4 9 7 6 1 2 5 0 6 1 1 0 6 6 8 9 5 6 2 6 6 3 3
 2 5 0 4 1 3 4 6 7 5 5 8 7 4 0 5 3 6 1 9 0 4 1 6 0 5 7 9 3 7 4 2 2 8 3 0 3
 7 1 7 3 6 4 1 7 8 1 5 4 2 0 6 1 5 4 9 9 2 9 7 9 7 8 6 5 2 0 7 7 6 6 6 3 5
 8 3 5 4 9 6 6 0 5 3 7 2 8 7 7 0 6 9 3 8 9 3 0 4 9 2 7 1 1 4 0 5 2 9 0 3 3
 8 8 1 8 9 9 3 7 4 6 6 7 1 9 7 1 6 7 7 7 1 2 3 3 1 7 4 9 8 9 5 0 8 6 5 6 1
 3 7 0 9 8 4 3 9 2 2 1 5 9 3 8 6 4 8 4 3 2 8 5 7 1 8 1 6 4 0 6 0 4 2 3 1 9
 9 6 2 5 8 0 9 9 1 3 8 6 5 1 9 4 3 6 7 9 1 8 7 1 7 2 8 0 2 3 1 3 5 1 4 3 5
 6 9 7 1 0 3 7 2 4 9 8 6 

0.872